In [1]:
import os
import polars as pl

os.chdir("C:\\Users\\ndgig\\Repositories\\embed_gram")
print(os.getcwd())

%load_ext autoreload
%autoreload 2

from embed_gram.embed_gram import NgramEncoder

C:\Users\ndgig\Repositories\embed_gram


In [2]:
splits = {
    "train": "data/train-*-of-*.parquet",
    "validation": "data/validation-00000-of-00001-869c898b519ad725.parquet",
}
docs = (
    pl.read_parquet("hf://datasets/roneneldan/TinyStories/" + splits["validation"])
    .sample(1000, seed=32)
    .to_pandas()
    .squeeze()
    .to_list()
)
docs

['John was walking down the street when he saw a vendor. He was so excited! He wanted to buy something, but his mommy said he had to wait. He watched the vendor with big eyes, but the vendor started to ignore him. John started to feel sad until he saw the modern vendor down the street. The vendor was dressed in bright colors and had so many interesting things to buy. \nJohn ran to the vendor as fast as he could and asked his mommy if he could buy something. His mommy said yes, and John was so happy! He went to the vendor, who gave him a big smile, and he chose a fun toy. \nJohn thanked the vendor, and skipped happily all the way home with his new toy, forgetting all about the vendor who chose to ignore him.',
 "Once upon a time, there was a kind lady named Mrs. Smith. She loved to watch cartoons on her TV. One day, she went to the store to buy some toys for the children in her neighborhood. She was very generous and wanted to make the kids happy.\n\nAt the store, she found a box of puz

In [3]:
if "ng" in locals():
    del ng
ng = NgramEncoder("sentence-transformers/all-mpnet-base-v2", device="cuda")
ng

In [4]:
ngrams, ngram_vecs = ng.encode_extract(docs, batch_size=256)
ngram_vecs

Encoding:   0%|          | 0/4 [00:00<?, ?it/s]

Extracting:   0%|          | 0/1000 [00:00<?, ?it/s]

[array([[-0.0214927 ,  0.16321369, -0.06586637, ...,  0.08002032,
          0.22443752, -0.10549764],
        [-0.01109547,  0.13306057, -0.08272329, ...,  0.05196882,
          0.1720919 , -0.10306581],
        [-0.04396629,  0.11845208, -0.11779586, ..., -0.01105932,
          0.11882553, -0.10801544],
        ...,
        [ 0.0229042 ,  0.18628055, -0.09708378, ...,  0.1496164 ,
          0.09831434, -0.08210539],
        [-0.01525108,  0.32037994, -0.11943443, ...,  0.10424803,
          0.1254838 , -0.08104426],
        [-0.02994183,  0.21989585, -0.07686567, ...,  0.04925682,
          0.04623343, -0.0910188 ]], dtype=float32),
 array([[-0.0130972 , -0.00555478,  0.05622481, ...,  0.18609092,
          0.04669908, -0.1167512 ],
        [ 0.00225209,  0.03993184,  0.04109003, ...,  0.16789322,
          0.04930646, -0.10876567],
        [-0.01577522,  0.06819918,  0.07119957, ...,  0.18746436,
          0.06879299, -0.06985482],
        ...,
        [-0.06169651,  0.19378985, -0.0

In [5]:
ngrams[0][:5], ngram_vecs[0][:5]

(['<s> john was',
  'john was walking',
  'was walking down',
  'walking down the',
  'down the street'],
 array([[-0.0214927 ,  0.16321369, -0.06586637, ...,  0.08002032,
          0.22443752, -0.10549764],
        [-0.01109547,  0.13306057, -0.08272329, ...,  0.05196882,
          0.1720919 , -0.10306581],
        [-0.04396629,  0.11845208, -0.11779586, ..., -0.01105932,
          0.11882553, -0.10801544],
        [-0.01604577,  0.16556884, -0.13966322, ..., -0.00274223,
          0.10229071, -0.0939441 ],
        [ 0.01288172,  0.2941607 , -0.10192159, ..., -0.02350864,
          0.09371153, -0.06995153]], dtype=float32))

In [10]:
queries = ng.encode_queries(["walked down the street", "bought from the shopkeeper"])
queries

Encoding:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting:   0%|          | 0/2 [00:00<?, ?it/s]

array([[ 0.06655846,  0.13982221, -0.03756061, ...,  0.05881256,
        -0.14844401, -0.11943534],
       [ 0.19894406, -0.06213467, -0.01230061, ...,  0.06223627,
        -0.01281988, -0.11989993]], dtype=float32)

In [11]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, metric="cosine")
nn.fit(ngram_vecs[0])
dists, idx = nn.kneighbors(queries)
[ngrams[0][i] for i in idx[1]]

['saw a vendor',
 'a vendor.',
 'saw a vendor.',
 'he saw a vendor',
 'to buy something',
 'buy something,',
 'to buy something,',
 'he saw a vendor.',
 'a vendor. he',
 'wanted to buy something']

In [8]:
[ngrams[0][i] for i in idx[1]]

['a vendor.',
 'saw a vendor',
 'saw a vendor.',
 'he saw a vendor',
 'a vendor. he',
 'vendor was dressed',
 'he saw a vendor.',
 'modern vendor down',
 'vendor. he',
 'saw a vendor. he']

In [9]:
ngrams[0]

['<s> john was',
 'john was walking',
 'was walking down',
 'walking down the',
 'down the street',
 'the street when',
 'street when he',
 'when he saw',
 'he saw a',
 'saw a vendor',
 'a vendor.',
 'vendor. he',
 '. he was',
 'he was so',
 'was so excited',
 'so excited!',
 'excited! he',
 '! he wanted',
 'he wanted to',
 'wanted to buy',
 'to buy something',
 'buy something,',
 'something, but',
 ', but his',
 'but his mommy',
 'his mommy said',
 'mommy said he',
 'said he had',
 'he had to',
 'had to wait',
 'to wait.',
 'wait. he',
 '. he watched',
 'he watched the',
 'watched the vendor',
 'the vendor with',
 'vendor with big',
 'with big eyes',
 'big eyes,',
 'eyes, but',
 ', but the',
 'but the vendor',
 'the vendor started',
 'vendor started to',
 'started to ignore',
 'to ignore him',
 'ignore him.',
 'him. john',
 '. john started',
 'john started to',
 'started to feel',
 'to feel sad',
 'feel sad until',
 'sad until he',
 'until he saw',
 'he saw the',
 'saw the modern',
 '